In [ ]:
import bolift
import numpy as np
import matplotlib.pyplot as plt
import json
import pandas as pd
from langchain.prompts.prompt import PromptTemplate


data_path = "paper/data/C2_yield_meth_oxy.csv"
np.random.seed(0)

In [ ]:
raw_data = pd.read_csv(data_path)
raw_data.head()

In [ ]:
asktell = bolift.AskTellFewShotMulti(
    x_formatter=lambda x: f"synthesis procedure:\n{x}",
    y_name="yield",
    y_formatter=lambda y: f"{y:.2f}",
    model="text-davinci-003",
    selector_k=4,
)

In [ ]:
train = np.random.choice(raw_data.shape[0], 450, replace=False)
test = np.setdiff1d(np.arange(raw_data.shape[0]), train)
# shuffle test
np.random.shuffle(test)
for i in train:
    asktell.tell(raw_data.iloc[i, 0], float(raw_data.iloc[i, 1]))
y = []
yhat = []
for j in test[:10]:
    y.append(float(raw_data.iloc[j, 1]))
    yhat.append(asktell.predict(raw_data.iloc[j, 0]))

In [ ]:
# filter out failed
y_filter = [yi for yi, yhi in zip(y, yhat) if len(yhi.values) > 0]
yhat_filter = [yhi for yi, yhi in zip(y, yhat) if len(yhi.values) > 0]

In [ ]:
modes = [yhat_filter[i].mode() for i in range(len(yhat_filter))]
std = [yhat_filter[i].std() for i in range(len(yhat_filter))]
plt.errorbar(y_filter, modes, yerr=std, fmt="o")
plt.plot(y, y)
plt.xlabel("Actual Yield")
plt.ylabel("Predicted Yield")
plt.show()

In [ ]:
for yh, yi in zip(yhat, y):
    print(yh, yi)